# My hangman file

In [2]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


In [10]:
random.choice(full_dict)

'unregainable'

So, I think the best way to do this would be to make a decision tree from scratch, but I think there is an issue of outliers that I am not super sure how to deal with. I think the dictionary is big enough that there are some words that the algorithm should figure out how to throw out. Like if there are 2 moves left, it should make the highest probability assumption it can which eliminates trees with more than 2 moves of surprise.

So... I think how the algorithm should work is, it doesnt just pick the ONE letter that would maximize the entropy next, it should find like a PATH of guesses that maximally reduces the entropy.

I am assuming there are always 6 tries, which seems a bit low...

It seems a bit complex though... If I build a tree I don't have to constantly re-calculate it, but I just move from node to node.

HOW DO I MAKE A TREE WITH NO LABELS THOUGH?

In [5]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

For testing purposes, I want to just remake hangman here so I can mess with it. So gut all of the connection to some external anything. I think I know how it works from running it a bit.

In [97]:
class MyHangman(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)


    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # The extra space characters must be coming from the website...
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        clean_word = word.replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word
        
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self but I don't think its necessary and this is just a hack thing anyways

        tries_remains = 6 #I think this needs to just be a magic number?
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                print("Success! the word was: %s" % word)
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            print("You Lose, the answer was: %s" % self.correct_word)
                


            



        """""           
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
    """""
        

In [52]:
word = "_____"
correctWord = "apple"
letter = "z"
#I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
indices = [i for i, x in enumerate(list(correctWord)) if x == letter]
wordList = list(word)
for i in indices:
    wordList[i] = letter #now I replace the occurrences of

word = "".join(wordList)

#while lidx != -1:
#    word[lidx] = letter
#    lidx = correctWord.find(letter)
print(word)

_____


In [101]:
game = MyHangman()
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: _________.
Guessing letter: e
Got a Letter, e, # of tries remaining: 6. Word: e_______e.
Guessing letter: a
Got a Letter, a, # of tries remaining: 6. Word: e___a___e.
Guessing letter: t
Incorrect guess t, # of tries remaining: 6. Word: e___a___e.
Guessing letter: i
Incorrect guess i, # of tries remaining: 5. Word: e___a___e.
Guessing letter: n
Got a Letter, n, # of tries remaining: 4. Word: e___an__e.
Guessing letter: l
Got a Letter, l, # of tries remaining: 4. Word: e___an_le.
Guessing letter: g
Got a Letter, g, # of tries remaining: 4. Word: e___angle.
Guessing letter: r
Got a Letter, r, # of tries remaining: 4. Word: e__rangle.
Guessing letter: m
Got a Letter, m, # of tries remaining: 4. Word: em_rangle.
Guessing letter: b
Success! the word was: embrangle


Ugh, after looking at this, I need to modify the MyHangman code to get some data from it right?

Ok... finally I can start writing the strategy... that shouldn't be too difficult right (uhhh)



My understanding is that there are 2 ways to do this... one that is the theoretical best strategy and the other that is just a good heuristic.

https://cs.stackexchange.com/questions/128667/what-is-the-optimal-algorithm-for-playing-the-hangman-word-game

The good one is to look at the game like a decision tree, see which choices have the highest entropy and make those. Of course there would be complications like you plan your route through the tree, and you have to figure out which nodes should be inaccessible?

The other is to get a game state function and use it to construct a matrix and then minimize it? I don't really get it right now. This uses dynamic programming, which is optimization over plain recursion?

So... let's say the word is $apple$, and look at a bit...

first prune dict to 5 letter words

In [105]:
corr_word = "apple"
word = "_____"
guessed_letters = []
full_dictionary_common_letter_sorted = collections.Counter("".join(full_dict)).most_common()


###############################################
# Replace with your own "guess" function here #
###############################################

# clean the word so that we strip away the space characters
# replace "_" with "." as "." indicates any character in regular expressions
clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
clean_word = word.replace("_",".")
print(word)
#print(word[::2])
print(clean_word)

# find length of passed word
len_word = len(clean_word)

# grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
current_dictionary = full_dict

new_dictionary = []

# iterate through all of the words in the old plausible dictionary
for dict_word in current_dictionary:
    # continue if the word is not of the appropriate length
    if len(dict_word) != len_word:
        continue
        
    # if dictionary word is a possible match then add it to the current dictionary
    if re.match(clean_word,dict_word):
        new_dictionary.append(dict_word)

# overwrite old possible words dictionary with updated version
current_dictionary = new_dictionary


# count occurrence of all characters in possible word matches
full_dict_string = "".join(new_dictionary)

c = collections.Counter(full_dict_string)
sorted_letter_count = c.most_common()                   

guess_letter = '!'

# return most frequently occurring letter in all possible words that hasn't been guessed yet
for letter,instance_count in sorted_letter_count:
    if letter not in guessed_letters:
        guess_letter = letter
        break
    
# if no word matches in training dictionary, default back to ordering of full dictionary
if guess_letter == '!':
    sorted_letter_count = full_dictionary_common_letter_sorted
    for letter,instance_count in sorted_letter_count:
        if letter not in guessed_letters:
            guess_letter = letter
            break            

#return guess_letter
print(guess_letter)


_____
.....
a
